In [6]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time
import numpy as np
import argparse, pprint
import matplotlib

from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

from src.UVMappingPipeline import UVMappingPipeline
from utils.sys_utils import *


pipe = UVMappingPipeline()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
%reload_ext autoreload
plt.close('all')
# tdir = "/home/hunter/data/images/obstacles"
# tdir = "/home/hyoung/data/images/obstacles"
tdir = homeDir + "/test/iai/obstacles"

found_imgs, _ = grab_dir_images(tdir)


p = interactive(pipe.pipeline,
    _img = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    ),
    line_method = {'Standard Hough Transform':0,'Probabilistic Hough Transform':1},
    input_method = {'Classic Thresholded':0,'Morphed Thresholded':1,'Canny Edges':2},
    e1 = (1,255,1), e2 = (1,255,1),
    ang = (0,360,1), rho = (1,1000,1),
    minLineLength = (0,255,1), maxLineGap = (0,255,1),
    houghThresh = (0,255,1), greyThreshU = (0,255,1), greyThreshV = (0,255,1), cnt_thresh = (0,5000,100),
    show_helpers = True, use_umap = True, flip_thresh_bin = False, norm_grey_thresh = (0.0,1.0,0.01)
)


row1 = HBox([p.children[0]])
row2 = HBox([p.children[1], p.children[2]], layout = Layout(flex_flow='row wrap'))
row3 = HBox([p.children[3], p.children[4], p.children[9]], layout = Layout(flex_flow='row wrap'))
row4 = HBox([p.children[5], p.children[6], p.children[7], p.children[8]], layout = Layout(display='flex',flex_flow='row'))

row5 = HBox([p.children[10], p.children[11], p.children[12], ], layout = Layout(flex_flow='row wrap'))
row6 = HBox([p.children[16]], layout = Layout(flex_flow='row wrap'))
row7 = HBox([p.children[13], p.children[14], p.children[15]], layout = Layout(flex_flow='row wrap'))
controls = VBox([row1,row2,row3,row4,row5,row6,row7], layout = Layout(display='flex'))
output = p.children[-1]
display(VBox([controls, output]))

VkJveChjaGlsZHJlbj0oVkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oU2VsZWN0KGRlc2NyaXB0aW9uPXUnRmlsZTonLCBsYXlvdXQ9TGF5b3V0KHdpZHRoPXUnMTAwJScpLCBvcHRpb27igKY=


In [ ]:
# im1 = pipe.umap; im2 = pipe.testU
im1 = pipe.vmap; im2 = pipe.testV

plt.figure(6)
plt.imshow(im1,'gray')
plt.show()

plt.figure(7)
plt.imshow(cv2.cvtColor(im2, cv2.COLOR_GRAY2BGR))
plt.show()

In [ ]:
tmp1 = np.copy(pipe.vmap); tmp0 = cv2.cvtColor(tmp1,cv2.COLOR_GRAY2BGR)

normImg = (tmp1 - np.mean(tmp1))/np.std(tmp1)

dmax_norm = np.max(normImg)
dmin_norm = np.min(normImg)
test = cv2.normalize(normImg,None,0,1,cv2.NORM_MINMAX)
m = 255.0/(dmax_norm-dmin_norm)
print(dmin_norm,dmax_norm,m)
tmp = cv2.convertScaleAbs(test, test)#, m)#,np.mean(tmp))
print(np.min(tmp),np.max(tmp),np.mean(tmp))

fig1 = plt.figure(6)
fig1.canvas.set_window_title('Custom Filter Input')
plt.imshow(pipe.vmap,cmap='gray')
plt.show()

iters = 3; kSz = (1,6)
thresh = [5, 10, 20, 30]
filters = [cv2.THRESH_BINARY,cv2.THRESH_BINARY_INV,cv2.THRESH_TRUNC+cv2.THRESH_BINARY_INV,cv2.THRESH_TOZERO]
kernelC = cv2.getStructuringElement(cv2.MORPH_CROSS,kSz)
kernelE = cv2.getStructuringElement(cv2.MORPH_RECT,(1,15))

_, grey = cv2.threshold(tmp1,5,255,filters[0])#+cv2.THRESH_TRUNC)

# gradient = cv2.morphologyEx(grey, cv2.MORPH_CLOSE, kSz)
gradient = cv2.morphologyEx(grey, cv2.MORPH_OPEN, kSz)

# tmpKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,kSz)
tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,kSz)

gradient2 = cv2.erode(tmp0,kernelE,iterations = 1)
gradient2 = cv2.dilate(gradient2,tmpKernel,iterations = 1)
# preMorph = cv2.erode(grey,kernelE,iterations = 3)
# preMorph = cv2.dilate(grey,tmpKernel,iterations = 3)


# # morph = cv2.morphologyEx(grey,cv2.MORPH_CLOSE,(1,2), iterations = 2)
# morph = cv2.morphologyEx(grey,cv2.MORPH_OPEN,(2,3), iterations = 1)

# _,preMorph = cv2.threshold(preMorph,1,255,cv2.THRESH_BINARY_INV)
# tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,15))
# preMorph = cv2.dilate(preMorph,tmpKernel,iterations = 3)
# tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(10,3))
# preMorph = cv2.erode(preMorph,kernelE,iterations = 2)
# print(preMorph.shape)

# mask = cv2.cvtColor(preMorph,cv2.COLOR_BGR2GRAY)
# res = cv2.bitwise_and(tmp,tmp, mask=mask)
_, res = cv2.threshold(gradient2,5,255,cv2.THRESH_BINARY)

out = res

fig2 = plt.figure(7)
fig2.canvas.set_window_title('Custom Filter Output')
plt.imshow(out,cmap='gray')
plt.show()

In [ ]:
print("""
POTENTIALLY GOOD VMAP FILTERING
""")

tmp1 = np.copy(pipe.vmap); tmp0 = cv2.cvtColor(tmp1,cv2.COLOR_GRAY2BGR)

kSz = (1,6)
kernelE = cv2.getStructuringElement(cv2.MORPH_RECT,(1,15))
_, grey = cv2.threshold(tmp1,5,255,cv2.THRESH_BINARY)

gradient = cv2.morphologyEx(grey, cv2.MORPH_OPEN, kSz)
tmpKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,kSz)

gradient2 = cv2.erode(tmp0,kernelE,iterations = 1)
gradient2 = cv2.dilate(gradient2,tmpKernel,iterations = 1)
_, res = cv2.threshold(gradient2,5,255,cv2.THRESH_BINARY)

out = res

In [ ]:
orig = np.copy(pipe.img)

imgMean = np.mean(orig)
imgStd = np.std(orig)
dmax_int = np.max(orig)
print('Original Image', imgMean, imgStd, dmax_int)

t0 = time.time()
normImg = (orig - np.mean(orig))/np.std(orig)
t1 = time.time()
dt1 = t1 - t0

# normImg = np.uint8(normImg)

dmax_norm1 = np.max(normImg)
dmin_norm1 = np.min(normImg)

m = 255.0/(dmax_norm1-dmin_norm1)
# m = 255.0/(dmax_norm1)
print(dmin_norm1,dmax_norm1,m)
# print(np.ceil(m))
normImg = cv2.convertScaleAbs(normImg, normImg, m,imgMean)
normMean = np.mean(normImg)
normStd = np.std(normImg)

print('Standardized Image #1', normMean, normStd, np.max(normImg), dt1)

t0 = time.time()
normImg2 = cv2.normalize(orig, orig, 0, 255, cv2.NORM_MINMAX)
t1 = time.time()
dt2 = t1 - t0

normMean2 = np.mean(normImg2)
normStd2 = np.std(normImg2)

print('Standardized Image #2', normMean2, normStd2, np.max(normImg2), dt2)

# umap,vmap,overlay,normImg  = get_uv_map(orig)
umap,vmap,overlay  = pipe.get_uv_map(normImg)

In [ ]:
plt.figure(6)
plt.imshow(orig,'gray')
plt.show()

plt.figure(7)
plt.imshow(normImg, 'gray')
plt.show()

plt.figure(8)
plt.imshow(normImg2, 'gray')
plt.show()

plt.figure(9)
plt.imshow(umap, 'gray')
plt.show()

In [ ]:
# print(pipe.disparityBounds[obsId],pipe.xBounds[obsId])

tmp = np.copy(pipe.greyV)
tmp2 = np.copy(pipe.img)
tmp2 = cv2.cvtColor(tmp2,cv2.COLOR_GRAY2BGR)
nObs = len(pipe.disparityBounds)
print(nObs)
b = 2; d = np.ceil(nObs / float(b))

plt.close(7)
fig, axes = plt.subplots(nrows=b, ncols=int(d), num=7)
obstacleImgs = []
for i in range(nObs):
    tmp2 = np.copy(pipe.img)
    tmp2 = cv2.cvtColor(tmp2,cv2.COLOR_GRAY2BGR)
    try:
        xs = pipe.xBounds[i]
        ys,disp = pipe.find_obstacles(tmp,pipe.disparityBounds[i], _threshold=30, lower_threshold=1, verbose = True)
        cv2.rectangle(tmp2,(xs[0],ys[0]),(xs[1],ys[-1]),(150,0,0),1)
    except:
        pass
    fig.axes[i].imshow(disp)
    obstacleImgs.append(tmp2)
plt.show()

In [ ]:
%reload_ext autoreload
# plt.figure(7)
h,w = pipe.vmap.shape[:2]
vmask = np.zeros((h,w,3), dtype=np.uint8)
cv2.rectangle(vmask,(0,0),(5,h),(255,255,255), cv2.FILLED)

mask = cv2.cvtColor(vmask,cv2.COLOR_BGR2GRAY)
mask_inv = cv2.bitwise_not(mask)

tmpV = np.copy(pipe.vmap)
tmpV = cv2.cvtColor(tmpV,cv2.COLOR_GRAY2BGR)
_, greyV = cv2.threshold(tmpV,11,255,cv2.THRESH_BINARY)

maskV, invMaskV = pipe.get_vmap_mask(greyV,verbose=True)

VMASK = cv2.bitwise_or(maskV, mask)
cv2.rectangle(invMaskV,(0,0),(5,h),(0,0,0), cv2.FILLED)

res = cv2.bitwise_and(pipe.vmap, pipe.vmap, mask = invMaskV)



_, greyV = cv2.threshold(res,11,255,cv2.THRESH_BINARY)

# plt.imshow(greyV)

cntsV, hierV = pipe.find_contours(greyV,20.0)
pipe.contoursV = cntsV; pipe.contour_hierarchyV = hierV

_cnts, limits, ellipses, pts, centers = pipe.extract_contour_x_bounds(cntsV)
# x,y,w,h = cv2.boundingRect(cntsV[-3])
# print(x, x+w, y, y+h)
# cv2.rectangle(greyV,(x,y),(x+w,y+h),(150,0,0),1)

pipe.plot_contours(greyV,cntsV,ellipses,centers,pts)

# plt.show()

In [173]:
im = np.copy(pipe.img)
norm = (im - np.mean(im))/np.std(im)
h, w = norm.shape[:2]

umapOrig, vmapOrig,_ = pipe.get_uv_map(im,get_overlay=False)

plt.figure(0)
plt.imshow(im, cmap='gray')
plt.show()
plt.figure(1)
plt.imshow(norm, cmap='gray')
plt.show()

In [174]:
def make_uv_overlay(_img, umap, vmap):
    blank = np.ones((umap.shape[0],vmap.shape[1]),np.uint8)*255
    pt1 = np.concatenate((np.copy(_img), vmap), axis=1)
    pt2 = np.concatenate((umap,blank),axis=1)
    overlay = np.concatenate((pt1,pt2),axis=0)
    overlay = cv2.cvtColor(overlay,cv2.COLOR_GRAY2BGR)
    return overlay

In [175]:
def umapping(_img, nBins=255, use_normalized=False, use_alternate_mapping=True, verbose=True):
    img = np.copy(_img);                       imMean = np.mean(img);      imStd = np.std(img)
    norm = (img - np.mean(img))/np.std(img);   normMean = np.mean(norm);   normStd = np.std(norm)
    h, w = norm.shape[:2]
    
    dmin = np.min(img);        dmax = np.max(img)
    dminN = np.min(norm);      dmaxN = np.max(norm)

    if(use_normalized):
        dmap = norm
        ds = np.linspace(dminN, dmaxN,nBins+1)
        dLims = (dminN,dmaxN)
    elif(not use_normalized and use_alternate_mapping):
        dmap = img
        ds = 0; dLims = (0,1)
    else: 
        dmap = norm
        ds = np.linspace(dmin, dmax,nBins+1)
        dLims = (dmin,dmax)
    
    umap = np.zeros((nBins,w), dtype=np.uint8)
    for col in range(w):
        data = dmap[:,col]
        if(not use_normalized and use_alternate_mapping):
            mu = np.mean(data)
            std = np.std(data)
            if(std == 0.0): std = 1
            data = (data - mu)/float(std)
            mx = np.max(data)
            mn = np.min(data)
            # print("U-Map Scan [%d] --- Min, Max, Mean, Std: %.3f, %.3f, %.3f, %.3f" % (col, mn, mx, mu, std) )
            urow, edges = np.histogram(data,nBins)
        else:
            urow, edges = np.histogram(data,nBins, dLims)
        umap[:,col] = urow
        
    if(verbose):
        print("""------------------------------------\r\n[U-Mapping] Input Image (%d x %d):\r\n------------------------------------
   * Un-Normalized Disparity Limits      = (%d, %d)
   * Normalized Disparity Limits         = (%.3f, %.3f)
   * U-Map Disparity Limits              = (%.3f, %.3f)
   * Un-Normalized Statistics   (%s, std) = (%.3f, %.3f)
   * Normalized Statistics      (%s, std) = (%.3f, %.3f)
   * U-Map Statistics           (%s, std) = (%.3f, %.3f)"""
             % (h,w,dmin,dmax,dminN,dmaxN, np.min(umap),np.max(umap), u"\u03BC",imMean,imStd,u"\u03BC",normMean,normStd,u"\u03BC",np.mean(umap),np.std(umap)) )
        
    return umap

In [176]:
def vmapping(_img, nBins=255, use_normalized=False, use_alternate_mapping=True, verbose=True):
    img = np.copy(_img);                       imMean = np.mean(img);      imStd = np.std(img)
    norm = (img - np.mean(img))/np.std(img);   normMean = np.mean(norm);   normStd = np.std(norm)
    h, w = norm.shape[:2]
    
    dmin = np.min(img);        dmax = np.max(img)
    dminN = np.min(norm);      dmaxN = np.max(norm)
    
    if(use_normalized):
        dmap = norm
        ds = np.linspace(dminN, dmaxN,nBins+1)
        dLims = (dminN,dmaxN)
    elif(not use_normalized and use_alternate_mapping):
        dmap = img
        ds = 0; dLims = (0,1)
    else: 
        dmap = norm
        ds = np.linspace(dmin, dmax,nBins+1)
        dLims = (dmin,dmax)
        
    vmap = np.zeros((h,nBins), dtype=np.uint8)
    for row in range(h):
        data = dmap[row,:]
        if(not use_normalized and use_alternate_mapping):
            mu = np.mean(data)
            std = np.std(data)
#             if(std == 0.0): std = 1
            data = (data - mu)/float(std)
            mx = np.max(data)
            mn = np.min(data)
            print("V-Map Scan [%d] --- Min, Max, Mean, Std: %.3f, %.3f, %.3f, %.3f" % (row, mn, mx, mu, std) )
            ds = np.linspace(dminN, dmaxN,nBins+1)
            vcol, edges = np.histogram(data,nBins)#,ds,(dmin,dmax))
        else:
            vcol, edges = np.histogram(data,nBins, dLims)
        vmap[row,:] = vcol
        
    if(verbose):
        print("""------------------------------------\r\n[V-Mapping] Input Image (%d x %d):\r\n------------------------------------
   * Un-Normalized Disparity Limits      = (%d, %d)
   * Normalized Disparity Limits         = (%.3f, %.3f)
   * V-Map Disparity Limits              = (%.3f, %.3f)
   * Un-Normalized Statistics   (%s, std) = (%.3f, %.3f)
   * Normalized Statistics      (%s, std) = (%.3f, %.3f)
   * V-Map Statistics           (%s, std) = (%.3f, %.3f)"""
             % (h,w,dmin,dmax,dminN,dmaxN, np.min(vmap),np.max(vmap), u"\u03BC",imMean,imStd,u"\u03BC",normMean,normStd,u"\u03BC",np.mean(vmap),np.std(vmap)) )
    
    print(edges)
    return vmap

In [189]:
timg = np.copy(pipe.img)
umap = umapping(timg)
vmap = vmapping(timg)

umapOrig, vmapOrig,_ = pipe.get_uv_map(timg,get_overlay=False)
# orig = umapOrig;     mapping = umap
orig = vmapOrig;     mapping = vmap

overlay = make_uv_overlay(timg,umap,vmap)

plt.figure(2)
plt.imshow(orig)
plt.show()
plt.figure(3)
plt.imshow(mapping)
plt.show()

plt.figure(4)
plt.imshow(overlay)
plt.show()

# plt.figure(2)
# plt.imshow(umapOrig, cmap='gray')
# plt.show()
# plt.figure(3)
# plt.imshow(umap, cmap='gray')
# plt.show()

------------------------------------
[U-Mapping] Input Image (480 x 640):
------------------------------------
   * Un-Normalized Disparity Limits      = (0, 255)
   * Normalized Disparity Limits         = (-0.872, 1.715)
   * U-Map Disparity Limits              = (0.000, 239.000)
   * Un-Normalized Statistics   (μ, std) = (85.934, 98.591)
   * Normalized Statistics      (μ, std) = (-0.000, 1.000)
   * U-Map Statistics           (μ, std) = (1.824, 8.883)
V-Map Scan [0] --- Min, Max, Mean, Std: -0.381, 5.826, 15.669, 41.080
V-Map Scan [1] --- Min, Max, Mean, Std: -0.439, 4.244, 23.928, 54.449
V-Map Scan [2] --- Min, Max, Mean, Std: -0.476, 3.817, 28.272, 59.398
V-Map Scan [3] --- Min, Max, Mean, Std: -0.488, 3.830, 28.805, 59.057
V-Map Scan [4] --- Min, Max, Mean, Std: -0.493, 4.574, 24.802, 50.331
V-Map Scan [5] --- Min, Max, Mean, Std: -0.494, 3.789, 29.433, 59.539
V-Map Scan [6] --- Min, Max, Mean, Std: -0.507, 3.254, 34.372, 67.809
V-Map Scan [7] --- Min, Max, Mean, Std: -0.525, 3.0

In [ ]:
def thresholding(_img, e1=1, e2=5,tmpThresh=3,greyThreshU=5,greyThreshV=5,cnt_thresh=50.0,
                 show_helpers=True,use_umap=True, flip_thresh_bin=False, norm_grey_thresh=0.5
):
    img = cv2.imread(_img)
    
    plt.figure(0)
    plt.imshow(img, cmap='gray')
    plt.show()
    
    umap, vmap, _ = pipe.get_uv_map(img, get_overlay=False)
    h,w = umap.shape[:2]
    clone = np.copy(umap)
    cv2.rectangle(clone,(0,0),(w,10),(0,0,0), cv2.FILLED)
    cv2.rectangle(clone,(0,h-10),(w,h),(0,0,0), cv2.FILLED)
    print(np.max(clone), np.min(clone))
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(e1,e2))
    
    _,tmp = cv2.threshold(clone,greyThreshU,np.max(clone),cv2.THRESH_TOZERO)
#     cv2.rectangle(tmp,(0,0),(w,10),(0,0,0), cv2.FILLED)
#     cv2.rectangle(tmp,(0,h-10),(w,h),(0,0,0), cv2.FILLED)
    opening = cv2.morphologyEx(tmp, cv2.MORPH_CLOSE, kernel)

    _, contours, hierarchy = cv2.findContours(opening,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    perimeters = [cv2.arcLength(cnt,True) for cnt in contours]
        
    filtered_cnts = [cnt for cnt in contours if cv2.arcLength(cnt,True) >= cnt_thresh]
    filtered_perimeters = [cv2.arcLength(cnt,True) for cnt in filtered_cnts]
#     print("Raw Contour Perimeters:",perimeters)
    print("Filtered Contour Perimeters:",filtered_perimeters)
        
    display = cv2.cvtColor(opening, cv2.COLOR_GRAY2BGR)
    for cnt in filtered_cnts:
        cv2.drawContours(display, [cnt], 0, (255,0,0), 2)
        
    
    
    plt.figure(1)
    plt.imshow(opening, cmap='gray')
    plt.show()
    
    plt.figure(2)
    plt.imshow(display, cmap='gray')
    plt.show()

In [ ]:
ttt = interactive(thresholding,_img = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    ), e1 = (1,255,1), e2 = (1,255,1),
    tmpThresh = (0,255,1), greyThreshU = (0.0,255.0,0.1), greyThreshV = (0,255,1), cnt_thresh = (0,5000,100),
    show_helpers = True, use_umap = True, flip_thresh_bin = False, norm_grey_thresh = (0.0,1.0,0.01)
)
display(ttt)